In [ ]:
import scan_csv
from LabelGenerator import *
from client import OIDClient
from FastSlidingWindow import *
from Util import *

ld = scan_csv.open_dicts()
train_image_codec = ld['train codec']

In [ ]:
print(ld.keys())

In [ ]:
car_images = ld['train class'][ld['class encodings']['Car']][0:10]
print(car_images[0])
print(car_images)

In [ ]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()

In [ ]:
def calc_iou_individual(pred_box, gt_box):
    pred_box[1] = (pred_box[1]+1.0)/2.0
    pred_box[3] = (pred_box[3]+1.0)/2.0
    
    x1_t = gt_box[0]-gt_box[1]/2.0
    x2_t = gt_box[0]+gt_box[1]/2.0
    
    y1_t = gt_box[2]-gt_box[3]/2.0
    y2_t = gt_box[2]+gt_box[3]/2.0
    
    x1_p = pred_box[0]-pred_box[1]/2.0
    x2_p = pred_box[0]+pred_box[1]/2.0
    
    y1_p = pred_box[2]-pred_box[3]/2.0
    y2_p = pred_box[2]+pred_box[3]/2.0
    
    if (x1_p > x2_p) or (y1_p > y2_p):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
    if (x1_t > x2_t) or (y1_t > y2_t):
        raise AssertionError(
            "Ground Truth box is malformed? true box: {}".format(gt_box))

    if (x2_t < x1_p or x2_p < x1_t or y2_t < y1_p or y2_p < y1_t):
        return 0.0

    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])

    inter_area = (far_x - near_x + 1) * (far_y - near_y + 1)
    true_box_area = (x2_t - x1_t + 1) * (y2_t - y1_t + 1)
    pred_box_area = (x2_p - x1_p + 1) * (y2_p - y1_p + 1)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    return iou

In [ ]:
client = OIDClient('192.168.1.31', 33333)

W_M_1 = tf.Variable(np.random.uniform(-1.0, 1.0, [4, 4, 3, 16]), dtype=tf.float32)

#W_M_2_UL = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)
W_M_2_U = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)
#W_M_2_UR = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)

W_M_2_L = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)
W_M_2 = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)
W_M_2_R = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)

#W_M_2_DL = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)
W_M_2_D = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)
#W_M_2_DR = tf.Variable(np.random.uniform(-1.0, 1.0, [3, 3, 16, 16]), dtype=tf.float32)

W_M_3 = tf.Variable(np.random.uniform(-1.0, 1.0, [25*25*16, 1*5]), dtype=tf.float32, name='WM3')

loss_value = tf.Variable(0.0, dtype=tf.float32)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

def mean_squared(inputs, targets):
  error = inputs - targets
  return tf.reduce_mean(tf.square(error))

for instance in car_images:
    X, Y, image, cmap = label_image(instance, train_image_codec, ld['class names'], client, 45, 50)
    
    if image is not None:
        X_pad = np.zeros([X.shape[0]+2, X.shape[1]+2, X.shape[2], X.shape[3], X.shape[4]], dtype=np.float32)
        X_pad[1:-1, 1:-1, :, :, :] = X
        
        X_encoded = np.zeros([X.shape[0]+2, X.shape[1]+2, 25, 25, 16], dtype=np.float32)
        
        # ONLY CAR
        Y = Y[:, :, 570:571, :]
        Y_pred = np.zeros(Y.shape, dtype=np.float32)
        
        iou_sum = 0.0
        iou_mean = 0.0
        loss = 0.0
        
        with tf.GradientTape() as tape:
            # ENCODE
            for i in range(1, X.shape[0]+1):
                for k in range(1, X.shape[1]+1):
                    conv1 = tf.nn.relu(tf.nn.conv2d([X_pad[i, k]], W_M_1, [1, 2, 2, 1], 'SAME'))
                    X_encoded[i, k] = conv1.numpy()[0]
            
            # NEIGHBOR PASS
            for n in range(3):
                X_encoded_temp = np.zeros(X_encoded.shape, dtype=X_encoded.dtype)
                for i in range(1, X.shape[0]+1):
                    for k in range(1, X.shape[1]+1):
                        #convUL = tf.nn.conv2d([X_encoded[i-1, k-1]], W_M_2_UL, [1, 1, 1, 1], 'SAME')
                        convU  = tf.nn.conv2d([X_encoded[i-1, k]], W_M_2_U, [1, 1, 1, 1], 'SAME')
                        #convUR = tf.nn.conv2d([X_encoded[i-1, k+1]], W_M_2_UR, [1, 1, 1, 1], 'SAME')
                        convL  = tf.nn.conv2d([X_encoded[i, k-1]], W_M_2_L, [1, 1, 1, 1], 'SAME')
                        conv   = tf.nn.conv2d([X_encoded[i, k]], W_M_2, [1, 1, 1, 1], 'SAME')
                        convR  = tf.nn.conv2d([X_encoded[i, k+1]], W_M_2_R, [1, 1, 1, 1], 'SAME')
                        #convDL = tf.nn.conv2d([X_encoded[i+1, k-1]], W_M_2_DL, [1, 2, 2, 1], 'SAME')
                        convD  = tf.nn.conv2d([X_encoded[i+1, k]], W_M_2_D, [1, 1, 1, 1], 'SAME')
                        #convDR = tf.nn.conv2d([X_encoded[i+1, k+1]], W_M_2_DR, [1, 2, 2, 1], 'SAME')
                        
                        new_val = tf.nn.relu(convU + convL + conv + convR + convD)
                        
                        X_encoded_temp[i, k] = new_val.numpy()[0]
                X_encoded = X_encoded_temp.copy()
            
            
            # PREDICT
            for i in range(1, X.shape[0]+1):
                for k in range(1, X.shape[1]+1):
                    patch_val = np.reshape(X_encoded[i, k], [1, 25*25*16])
                    pred = tf.nn.tanh(tf.matmul(patch_val, W_M_3))
                    Y_pred[i-1, k-1] = pred.numpy()
                    iou_sum += calc_iou_individual(pred[0, 1:].numpy(), Y[i-1, k-1, 0][1:])
            
            loss_value = mean_squared(Y_pred, Y)
            iou_mean = iou_sum / (Y_pred.shape[0]*Y_pred.shape[1])
            loss = 1.0 - iou_mean
        # [W_M_1, W_M_2, W_M_2_U, W_M_2_L, W_M_2_R, W_M_2_D, W_M_3]
        print(Y_pred.shape, ' iou_sum:', iou_sum, ' iou_mean:', iou_mean, ' LOSS:', loss)
        optimizer.apply_gradients(zip(tape.gradient(loss_value, [W_M_3]), [W_M_3]),
                            global_step=tf.train.get_or_create_global_step())
        
        img_orig = draw_from_label(image, Y, cmap)
        img_pred = draw_from_label(image, Y_pred, cmap)
        cv2.imshow('orig', img_orig)
        cv2.imshow('pred', img_pred)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
                   
                        